In [15]:

import os, sys
from pathlib import Path

REPO_URL = "https://github.com/BEE3600-2025/BEE-3600-Assets"
REPO_DIR = Path("/content/BEE-3600-Assets")
PROJECT_DIR = REPO_DIR / "CBB-butanol"

# Ensure /content
os.makedirs("/content", exist_ok=True)
os.chdir("/content")

# Clone or update
if not REPO_DIR.exists():
    print("Cloning repository...")
    !git clone -q {REPO_URL}
else:
    print("Repository exists; pulling latest...")
    %cd {REPO_DIR}
    !git pull -q
    %cd /content

# Git LFS (best effort)
%cd {REPO_DIR}
!git lfs version || true
!git lfs install --skip-repo || true
!git lfs pull || true

# Enter project
%cd {PROJECT_DIR}

# Put project subfolder and repo root at the FRONT of sys.path
for p in [str(PROJECT_DIR), str(REPO_DIR)]:
    if p in sys.path:
        sys.path.remove(p)
for p in reversed([str(PROJECT_DIR), str(REPO_DIR)]):
    sys.path.insert(0, p)

print("CWD:", os.getcwd())
print("PYTHONPATH head:", sys.path[:3])

# Dependencies
import pathlib
installed = False
for req in [pathlib.Path("requirements.txt"), pathlib.Path("../requirements.txt")]:
    if req.exists():
        print(f"Installing from {req} ...")
        !python -m pip -q install -r "{req}"
        installed = True
        break
if not installed:
    print("No requirements.txt found; installing a safe default stack...")
    !python -m pip -q install numpy pandas matplotlib scipy ipywidgets openpyxl biopython

print("Environment ready.")


Repository exists; pulling latest...
/content/BEE-3600-Assets
/content
/content/BEE-3600-Assets
git-lfs/3.0.2 (GitHub; linux amd64; go 1.18.1)
Git LFS initialized.
/content/BEE-3600-Assets/CBB-butanol
CWD: /content/BEE-3600-Assets/CBB-butanol
PYTHONPATH head: ['/content/BEE-3600-Assets/CBB-butanol', '/content/BEE-3600-Assets', '/content']
Installing from requirements.txt ...
Environment ready.


In [16]:
# Sanity check: ensure we're importing the correct 'utils' (from this repo)
import utils, inspect
print("utils module file:", getattr(utils, "__file__", None))

import importlib
balanceUtils = importlib.import_module("utils.balanceUtils")
print("balanceUtils file:", getattr(balanceUtils, "__file__", None))

# Show candidate function names so we can match the expected API
names = [n for n in dir(balanceUtils) if not n.startswith("_")]
print("Available in balanceUtils:", names[:50])

# If the expected function isn't present, fail fast with a helpful message
NEEDED = ["SolveFluxBalanceEquation","ConvertIndexedSMatrix","ImportIOStatus"]
missing = [n for n in NEEDED if not hasattr(balanceUtils, n)]
if missing:
    raise ImportError(f"balanceUtils is loaded from {balanceUtils.__file__}, but missing: {missing}. "
                      f"Choose from: {names[:50]} or adjust the import names to match the file.")

utils module file: /content/BEE-3600-Assets/CBB-butanol/utils/__init__.py
balanceUtils file: /content/BEE-3600-Assets/CBB-butanol/utils/balanceUtils.py
Available in balanceUtils: ['BalanceStoichiometricMatrix', 'CheckConcentrationChangeVector', 'ConvertIndexedSMatrix', 'ExportUniqueCompoundsWithIOStatus', 'FindTargetIndex', 'FindTargetIndexFailure', 'FindUniqueTerms', 'GenerateFileList', 'GenerateIOStatusList', 'GenerateIndexedSMatrixT', 'GenerateMergedIOStatusList', 'GenerateStoichiometricMatrix', 'GenerateUniqueCompoundsList', 'Get_IO_Status_for_Compound_List', 'ImportBalanceAndOutputStoichiometricMatrices', 'ImportBalanceAndReportStoichiometricMatrix', 'ImportBalanceAndReportStoichiometricMatrix2', 'ImportIOStatus', 'ImportReactionFile', 'ImportStoichiometricMatrix', 'ParseReactionList', 'PrintStoichiometry', 'SolveFluxBalanceEquation', 'UpdateSMatrixWithMultipliers', 'fmin_slsqp', 'ieqconstraint', 'matmul', 'scipy', 'w_rss1']


In [17]:
import shutil
import os

# Remove the conflicting /content/utils directory if it exists
conflicting_utils_path = '/content/utils'
if os.path.exists(conflicting_utils_path):
    print(f"Removing conflicting directory: {conflicting_utils_path}")
    shutil.rmtree(conflicting_utils_path)
else:
    print(f"Conflicting directory not found: {conflicting_utils_path}")

Conflicting directory not found: /content/utils


In [18]:
# Sanity check: ensure we're importing the correct 'utils' (from this repo)
# After removing the conflicting /content/utils, this should now correctly import from the project's utils.
from utils import balanceUtils
print("balanceUtils module file:", getattr(balanceUtils, "__file__", None))

# Show candidate function names so we can match the expected API
names = [n for n in dir(balanceUtils) if not n.startswith("_")]
print("Available in balanceUtils:", names[:50])

# If the expected function isn't present, fail fast with a helpful message
NEEDED = ["SolveFluxBalanceEquation","ConvertIndexedSMatrix","ImportIOStatus"]
missing = [n for n in NEEDED if not hasattr(balanceUtils, n)]
if missing:
    raise ImportError(f"balanceUtils is loaded from {balanceUtils.__file__}, but missing: {missing}. "
                      f"Choose from: {names[:50]} or adjust the import names to match the file.")
else:
    print("All expected functions found in balanceUtils.")

balanceUtils module file: /content/BEE-3600-Assets/CBB-butanol/utils/balanceUtils.py
Available in balanceUtils: ['BalanceStoichiometricMatrix', 'CheckConcentrationChangeVector', 'ConvertIndexedSMatrix', 'ExportUniqueCompoundsWithIOStatus', 'FindTargetIndex', 'FindTargetIndexFailure', 'FindUniqueTerms', 'GenerateFileList', 'GenerateIOStatusList', 'GenerateIndexedSMatrixT', 'GenerateMergedIOStatusList', 'GenerateStoichiometricMatrix', 'GenerateUniqueCompoundsList', 'Get_IO_Status_for_Compound_List', 'ImportBalanceAndOutputStoichiometricMatrices', 'ImportBalanceAndReportStoichiometricMatrix', 'ImportBalanceAndReportStoichiometricMatrix2', 'ImportIOStatus', 'ImportReactionFile', 'ImportStoichiometricMatrix', 'ParseReactionList', 'PrintStoichiometry', 'SolveFluxBalanceEquation', 'UpdateSMatrixWithMultipliers', 'fmin_slsqp', 'ieqconstraint', 'matmul', 'scipy', 'w_rss1']
All expected functions found in balanceUtils.


In [19]:
import re
import os.path
import pdb

from numpy import unique, int8, array

from utils.balanceUtils import SolveFluxBalanceEquation, ConvertIndexedSMatrix, ImportIOStatus, \
ImportReactionFile, GenerateIndexedSMatrixT, PrintStoichiometry, GenerateMergedIOStatusList, \
Get_IO_Status_for_Compound_List, ExportUniqueCompoundsWithIOStatus, GenerateIOStatusList, ParseReactionList

from utils.vectorOutput4 import generateOutputMatrixWithHeaders, writeOutputMatrix, Write_SMatrix
from utils.specutils12 import ensure_dir

# Reaction List

In [20]:
reactionList = \
['CO2 + Ribulose15BP + H2O -> 2*3PG', \
 '3PG + ATP -> 13BPG + ADP', \
 '13BPG + NADH + Hplus -> G3P + P + NADplus', \
 'G3P + GlyceronePhosph -> F16BP', \
 'F16BP + H2O -> F6P + P', \
 'F6P + G3P -> E4P + X5P', \
 'G3P -> GlyceronePhosph', \
 'E4P + GlyceronePhosph -> SH17BP', \
 'X5P -> Ribulose5P', \
 'SH17BP + H2O -> SH7P + P', \
 'SH7P + G3P -> Ribose5P + X5P', \
 'Ribose5P -> Ribulose5P', \
 'ATP + Ribulose5P -> ADP + Ribulose15BP', \
 '3PG -> 2PG', \
 '2PG -> PEP + H2O', \
 'ADP + PEP + P -> Pyruvate + ATP', \
 'NADplus + Pyruvate + CoA -> NADH + Acetyl-CoA + CO2 + 2*Hplus', \
 '2*Acetyl-CoA -> CoA + Acetoacetyl-CoA', \
 'Acetoacetyl-CoA + NADH + Hplus -> Hydroxybutanoyl-CoA + NADplus', \
 'Hydroxybutanoyl-CoA -> Crotonoyl-CoA + H2O', \
 'Crotonoyl-CoA + NADH + Hplus -> Butanoyl-CoA + NADplus', \
 'Butanoyl-CoA + NADH + Hplus -> Butanal + CoA + NADplus', \
 'Butanal + NADH + Hplus -> 1-Butanol + NADplus' \
]

In [21]:
len(reactionList)

23

In [22]:
reactionList[-8]

'ADP + PEP + P -> Pyruvate + ATP'

In [23]:
reactantsToGet = ['ATP', 'NADH', 'CO2']

# Prepare Stoichiometric Matrix

Figure out the unique compounds, reactions, and calculate a stoichiomietric matrix.

In [24]:
uniqueCompounds, reactions, sMatrixTKeyIndexed = \
ParseReactionList(reactionList, reactionArrow='->')

In [25]:
uniqueCompoundsIOStatusProposed = GenerateIOStatusList(uniqueCompounds)
uniqueCompoundsIOStatusProposed

[[np.str_('1-Butanol'), 'Intermediate'],
 [np.str_('13BPG'), 'Intermediate'],
 [np.str_('2PG'), 'Intermediate'],
 [np.str_('3PG'), 'Intermediate'],
 [np.str_('ADP'), 'Intermediate'],
 [np.str_('ATP'), 'Input'],
 [np.str_('Acetoacetyl-CoA'), 'Intermediate'],
 [np.str_('Acetyl-CoA'), 'Intermediate'],
 [np.str_('Butanal'), 'Intermediate'],
 [np.str_('Butanoyl-CoA'), 'Intermediate'],
 [np.str_('CO2'), 'Input/Output'],
 [np.str_('CoA'), 'Intermediate'],
 [np.str_('Crotonoyl-CoA'), 'Intermediate'],
 [np.str_('E4P'), 'Intermediate'],
 [np.str_('F16BP'), 'Intermediate'],
 [np.str_('F6P'), 'Intermediate'],
 [np.str_('G3P'), 'Intermediate'],
 [np.str_('GlyceronePhosph'), 'Intermediate'],
 [np.str_('H2O'), 'Input/Output'],
 [np.str_('Hplus'), 'Intermediate'],
 [np.str_('Hydroxybutanoyl-CoA'), 'Intermediate'],
 [np.str_('NADH'), 'Input'],
 [np.str_('NADplus'), 'Intermediate'],
 [np.str_('P'), 'Intermediate'],
 [np.str_('PEP'), 'Intermediate'],
 [np.str_('Pyruvate'), 'Intermediate'],
 [np.str_('Rib

# Edit This

Make your best guess about the input/output status of each compound. Input/Output, Input, and Output are all the same from the point of view of the code. The crucial difference is between Target (one allowed); Intermediate; and Input|Output.

In [26]:
uniqueCompoundsIOStatusEdit = \
[['1-Butanol', 'Target'],
 ['13BPG', 'Intermediate'],
 ['2PG', 'Intermediate'],
 ['3PG', 'Intermediate'],
 ['ADP', 'Output'],
 ['ATP', 'Input'],
 ['Acetoacetyl-CoA', 'Intermediate'],
 ['Acetyl-CoA', 'Intermediate'],
 ['Butanal', 'Intermediate'],
 ['Butanoyl-CoA', 'Intermediate'],
 ['CO2', 'Input/Output'],
 ['CoA', 'Intermediate'],
 ['Crotonoyl-CoA', 'Intermediate'],
 ['E4P', 'Intermediate'],
 ['F16BP', 'Intermediate'],
 ['F6P', 'Intermediate'],
 ['G3P', 'Intermediate'],
 ['GlyceronePhosph', 'Intermediate'],
 ['H2O', 'Input/Output'],
 ['Hplus', 'Output'],
 ['Hydroxybutanoyl-CoA', 'Intermediate'],
 ['NADH', 'Input'],
 ['NADplus', 'Output'],
 ['P', 'Output'],
 ['PEP', 'Intermediate'],
 ['Pyruvate', 'Intermediate'],
 ['Ribose5P', 'Intermediate'],
 ['Ribulose15BP', 'Intermediate'],
 ['Ribulose5P', 'Intermediate'],
 ['SH17BP', 'Intermediate'],
 ['SH7P', 'Intermediate'],
 ['X5P', 'Intermediate']]

i = 0
ioStatusEdit = []
while i < len(uniqueCompoundsIOStatusEdit):
    ioStatusEdit.append(uniqueCompoundsIOStatusEdit[i][1])
    i += 1

# Balance the Stoichiometric Matrix

In [27]:
sMatrixT = ConvertIndexedSMatrix(sMatrixTKeyIndexed, uniqueCompounds)
sMatrix = sMatrixT.transpose()


[fVectorOpt, cDotVectorOpt, cDotVectorOptNorm, result] = \
SolveFluxBalanceEquation(sMatrix, reactions, uniqueCompounds, ioStatusEdit)

In [28]:
[fVectorOpt, cDotVectorOpt, cDotVectorOptNorm, result];

In [29]:
i = 0
printEverything = True
while i < len(uniqueCompounds):
        ioStatus = ioStatusEdit[i]
        compound = uniqueCompounds[i]
        if printEverything == False and (compound in reactantsToGet) or ioStatus == 'Target':
            print(uniqueCompounds[i] + ':\t' + ioStatus + '\t' + "%.1f" % cDotVectorOptNorm[i])
        elif printEverything == True:
            print(uniqueCompounds[i] + ':\t' + ioStatus + '\t' + "%.1f" % cDotVectorOptNorm[i])
        i += 1

1-Butanol:	Target	1.0
13BPG:	Intermediate	-0.0
2PG:	Intermediate	-0.0
3PG:	Intermediate	-0.0
ADP:	Output	14.0
ATP:	Input	-14.0
Acetoacetyl-CoA:	Intermediate	0.0
Acetyl-CoA:	Intermediate	0.0
Butanal:	Intermediate	-0.0
Butanoyl-CoA:	Intermediate	-0.0
CO2:	Input/Output	-4.0
CoA:	Intermediate	-0.0
Crotonoyl-CoA:	Intermediate	-0.0
E4P:	Intermediate	-0.0
F16BP:	Intermediate	0.0
F6P:	Intermediate	-0.0
G3P:	Intermediate	-0.0
GlyceronePhosph:	Intermediate	0.0
H2O:	Input/Output	-7.0
Hplus:	Output	-10.0
Hydroxybutanoyl-CoA:	Intermediate	0.0
NADH:	Input	-12.0
NADplus:	Output	12.0
P:	Output	12.0
PEP:	Intermediate	-0.0
Pyruvate:	Intermediate	-0.0
Ribose5P:	Intermediate	0.0
Ribulose15BP:	Intermediate	-0.0
Ribulose5P:	Intermediate	0.0
SH17BP:	Intermediate	0.0
SH7P:	Intermediate	0.0
X5P:	Intermediate	0.0


In [30]:
fVectorOpt / fVectorOpt[-2]

array([ 6.00145773, 10.00399833, 10.00433024,  2.00049759,  2.0004781 ,
        2.00069611,  4.00259604,  2.00158413,  4.00108371,  2.00138922,
        2.00072924,  2.00040461,  6.00126611,  1.99912786,  1.9995726 ,
        2.00025092,  2.00040743,  1.00008536,  0.99904728,  0.99874093,
        0.99913728,  1.        ,  1.00031172])

In [31]:
sMatrix;

In [32]:
reactionList[-1]

'Butanal + NADH + Hplus -> 1-Butanol + NADplus'

# Task
Clear Python module cache for `utils` and re-import `balanceUtils` to ensure the correct module is loaded and all expected functions (`SolveFluxBalanceEquation`, `ConvertIndexedSMatrix`, `ImportIOStatus`) are available.

## Clear Python module cache and re-import utils.balanceUtils

### Subtask:
Clear any cached versions of the 'utils' and 'utils.balanceUtils' modules from the Python interpreter's memory and then re-import 'balanceUtils' to ensure the correct version from the project directory is loaded with all its expected functions.


**Reasoning**:
To ensure correct module loading, it is necessary to clear Python's `sys.modules` cache for 'utils' and 'utils.balanceUtils' before re-importing. This step modifies cell `44c3055f` as per the instructions to incorporate the necessary cache clearing logic and the `import sys` statement.



In [33]:
import sys

# Sanity check: ensure we're importing the correct 'utils' (from this repo)
# After removing the conflicting /content/utils, this should now correctly import from the project's utils.

# Clear module cache to ensure re-importing the correct version
if 'utils' in sys.modules:
    del sys.modules['utils']
if 'utils.balanceUtils' in sys.modules:
    del sys.modules['utils.balanceUtils']

from utils import balanceUtils
print("balanceUtils module file:", getattr(balanceUtils, "__file__", None))

# Show candidate function names so we can match the expected API
names = [n for n in dir(balanceUtils) if not n.startswith("_")]
print("Available in balanceUtils:", names[:50])

# If the expected function isn't present, fail fast with a helpful message
NEEDED = ["SolveFluxBalanceEquation","ConvertIndexedSMatrix","ImportIOStatus"]
missing = [n for n in NEEDED if not hasattr(balanceUtils, n)]
if missing:
    raise ImportError(f"balanceUtils is loaded from {balanceUtils.__file__}, but missing: {missing}. "
                      f"Choose from: {names[:50]} or adjust the import names to match the file.")
else:
    print("All expected functions found in balanceUtils.")

balanceUtils module file: /content/BEE-3600-Assets/CBB-butanol/utils/balanceUtils.py
Available in balanceUtils: ['BalanceStoichiometricMatrix', 'CheckConcentrationChangeVector', 'ConvertIndexedSMatrix', 'ExportUniqueCompoundsWithIOStatus', 'FindTargetIndex', 'FindTargetIndexFailure', 'FindUniqueTerms', 'GenerateFileList', 'GenerateIOStatusList', 'GenerateIndexedSMatrixT', 'GenerateMergedIOStatusList', 'GenerateStoichiometricMatrix', 'GenerateUniqueCompoundsList', 'Get_IO_Status_for_Compound_List', 'ImportBalanceAndOutputStoichiometricMatrices', 'ImportBalanceAndReportStoichiometricMatrix', 'ImportBalanceAndReportStoichiometricMatrix2', 'ImportIOStatus', 'ImportReactionFile', 'ImportStoichiometricMatrix', 'ParseReactionList', 'PrintStoichiometry', 'SolveFluxBalanceEquation', 'UpdateSMatrixWithMultipliers', 'fmin_slsqp', 'ieqconstraint', 'matmul', 'scipy', 'w_rss1']
All expected functions found in balanceUtils.


**Reasoning**:
The previous step successfully cleared the module cache and re-imported `balanceUtils`, confirming that all necessary functions are now available. The reported `SyntaxWarning` messages are minor issues within the imported library code that do not prevent the current notebook's execution and will be ignored for now. The next logical step is to import the remaining utility functions required for the subsequent calculations.



In [34]:
import re
import os.path
import pdb

from numpy import unique, int8, array

from utils.balanceUtils import SolveFluxBalanceEquation, ConvertIndexedSMatrix, ImportIOStatus, \
ImportReactionFile, GenerateIndexedSMatrixT, PrintStoichiometry, GenerateMergedIOStatusList, \
Get_IO_Status_for_Compound_List, ExportUniqueCompoundsWithIOStatus, GenerateIOStatusList, ParseReactionList

from utils.vectorOutput4 import generateOutputMatrixWithHeaders, writeOutputMatrix, Write_SMatrix
from utils.specutils12 import ensure_dir

**Reasoning**:
The previous step successfully cleared the module cache and re-imported `balanceUtils`, confirming that all necessary functions are now available. The reported `SyntaxWarning` messages are minor issues within the imported library code that do not prevent the current notebook's execution and will be ignored for now. The next logical step is to import the remaining utility functions required for the subsequent calculations.



In [35]:
import re
import os.path
import pdb

from numpy import unique, int8, array

from utils.balanceUtils import SolveFluxBalanceEquation, ConvertIndexedSMatrix, ImportIOStatus, \
ImportReactionFile, GenerateIndexedSMatrixT, PrintStoichiometry, GenerateMergedIOStatusList, \
Get_IO_Status_for_Compound_List, ExportUniqueCompoundsWithIOStatus, GenerateIOStatusList, ParseReactionList

from utils.vectorOutput4 import generateOutputMatrixWithHeaders, writeOutputMatrix, Write_SMatrix
from utils.specutils12 import ensure_dir

## Final Task

### Subtask:
Confirm that balanceUtils is correctly imported and all required functions are available.


## Summary:

### Q&A
Yes, the `balanceUtils` module was correctly re-imported after clearing the cache, and all expected functions (`SolveFluxBalanceEquation`, `ConvertIndexedSMatrix`, `ImportIOStatus`) were confirmed to be available.

### Data Analysis Key Findings
*   The Python module cache for `utils` and `utils.balanceUtils` was successfully cleared, enabling a fresh re-import of `balanceUtils`.
*   The `balanceUtils` module was confirmed to be loaded from the expected project directory (`/content/BEE-3600-Assets/CBB-butanol/utils/balanceUtils.py`), ensuring the correct version is in use.
*   All critical functions (`SolveFluxBalanceEquation`, `ConvertIndexedSMatrix`, `ImportIOStatus`) were found to be present and accessible within the re-imported `balanceUtils` module.
*   During the import process, several `SyntaxWarning` messages related to invalid escape sequences were observed in `balanceUtils.py` and `specutils12.py`, although these did not impede the successful loading and functionality of the modules.

### Insights or Next Steps
*   Investigate and resolve the `SyntaxWarning` messages in `balanceUtils.py` and `specutils12.py` to improve code quality and prevent potential future issues.
*   Given the successful validation, proceed with subsequent analysis steps that rely on the `balanceUtils` module and its functions.
